# 多層感知器Dropout實作

## 載入套件

In [1]:
import tensorflow as tf
import input_data
tf.set_random_seed(1234)

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 訓練資料

In [2]:
batch_xs, batch_ys = mnist.train.next_batch(5)

## 建立Computational Graph

In [3]:
x_ = tf.placeholder(tf.float32, [None, 784], name="x_")
y_ = tf.placeholder(tf.float32, [None, 10], name="y_")

W1 = tf.Variable(tf.truncated_normal([784,800], stddev=0.1), name="W1")
b1 = tf.Variable(tf.zeros([800]), name="b1")
h1 = tf.nn.sigmoid(tf.matmul(x_, W1) + b1)

W2 = tf.Variable(tf.truncated_normal([800,10], stddev=0.1), name="W2")
b2 = tf.Variable(tf.zeros([10]), name="b2")
y = tf.nn.softmax(tf.matmul(h1, W2) + b2)

cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

lambda_ = tf.placeholder(tf.float32, name="lambda")
regularizer = tf.reduce_sum(tf.square(W1))+tf.reduce_sum(tf.square(W2))

cost = cross_entropy + lambda_*regularizer


optimizer = tf.train.GradientDescentOptimizer(0.003)
trainer = optimizer.minimize(cost)

init = tf.initialize_all_variables()


## Session

In [4]:
sess = tf.Session()
sess.run(init)

## 建立成效評估用的Computational Graph

In [5]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Saver

In [6]:
saver = tf.train.Saver(max_to_keep=10)



## 訓練

In [7]:
i = 0
lambdas = [0,0.00001,0.0001,0.001,0.01,0.1]
for lamb in lambdas:
    patience = 50
    best_accurarcy = 0
    sess.run(W1.assign(tf.truncated_normal([784,800], stddev=0.1)))
    sess.run(b1.assign(tf.zeros([800])))
    sess.run(W2.assign(tf.truncated_normal([800,10], stddev=0.1)))
    sess.run(b2.assign(tf.zeros([10])))
    print "---------start----------"
    while True:
        i += 1
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(trainer,  feed_dict={x_: batch_xs, y_: batch_ys, lambda_:lamb})
        if i%20 == 0:
            train_accurarcy = sess.run(accuracy, 
                feed_dict={x_: batch_xs, y_: batch_ys, lambda_:lamb})
            valid_accurarcy = sess.run(accuracy, 
                feed_dict={x_: mnist.validation.images, y_: mnist.validation.labels, lambda_:lamb})
            print "lambda:%s, iter: %s, train: %s, valid: %s" %(lamb, i, train_accurarcy, valid_accurarcy)
            if valid_accurarcy > best_accurarcy:
                patience = 50
                best_accurarcy = valid_accurarcy
                print "save model"
                model_name = "model_regularization_%s.ckpt"%( str(lamb).replace(".","") )
                saver.save(sess, model_name)
            else:
                patience -= 1
                if patience == 0:
                    print "early stop"
                    break 

---------start----------
lambda:0, iter: 20, train: 0.72, valid: 0.6838
save model
lambda:0, iter: 40, train: 0.84, valid: 0.8234
save model
lambda:0, iter: 60, train: 0.9, valid: 0.8114
lambda:0, iter: 80, train: 0.81, valid: 0.849
save model
lambda:0, iter: 100, train: 0.94, valid: 0.8792
save model
lambda:0, iter: 120, train: 0.87, valid: 0.8574
lambda:0, iter: 140, train: 0.9, valid: 0.8776
lambda:0, iter: 160, train: 0.84, valid: 0.8734
lambda:0, iter: 180, train: 0.94, valid: 0.8906
save model
lambda:0, iter: 200, train: 0.88, valid: 0.877
lambda:0, iter: 220, train: 0.95, valid: 0.8916
save model
lambda:0, iter: 240, train: 0.94, valid: 0.89
lambda:0, iter: 260, train: 0.92, valid: 0.8624
lambda:0, iter: 280, train: 0.91, valid: 0.898
save model
lambda:0, iter: 300, train: 0.91, valid: 0.9034
save model
lambda:0, iter: 320, train: 0.96, valid: 0.896
lambda:0, iter: 340, train: 0.97, valid: 0.9032
lambda:0, iter: 360, train: 0.93, valid: 0.9074
save model
lambda:0, iter: 380, tra

In [10]:

for lamb in lambdas:
    model_name = "model_regularization_%s.ckpt"%( str(lamb).replace(".","") )
    saver.restore(sess, model_name)
    
    valid_accurarcy = sess.run(accuracy, feed_dict={x_: mnist.validation.images, y_: mnist.validation.labels, lambda_:lamb})
    test_accurarcy = sess.run(accuracy, feed_dict={x_: mnist.test.images, y_: mnist.test.labels, lambda_:lamb})
    print "lambda:%s valid:%s, test:%s"%(lamb, valid_accurarcy,test_accurarcy)

lambda:0 valid:0.9754, test:0.9731
lambda:1e-05 valid:0.975, test:0.9717
lambda:0.0001 valid:0.9774, test:0.9736
lambda:0.001 valid:0.976, test:0.9722
lambda:0.01 valid:0.9724, test:0.9683
lambda:0.1 valid:0.9204, test:0.9108
